In [70]:
import subprocess

# Lista dei file Python da eseguire
scripts = [
    "torch_qnn/q_ops.py"
    "torch_qnn/q_layers.py"
    "qlstm.py"
]

def run_scripts(scripts):
    for script in scripts:
        print(f"Eseguendo {script}...")
        try:
            # Esegui il file Python con python3
            subprocess.run(["python3", script], check=True)
            print(f"{script} eseguito con successo.")
        except subprocess.CalledProcessError as e:
            print(f"Errore nell'esecuzione di {script}: {e}")

if __name__ == "__main__":
    run_scripts(scripts)

Eseguendo /torch_qnn/q_ops.py/torch_qnn/q_layers.pyqlstm.py...
Errore nell'esecuzione di /torch_qnn/q_ops.py/torch_qnn/q_layers.pyqlstm.py: Command '['python3', '/torch_qnn/q_ops.py/torch_qnn/q_layers.pyqlstm.py']' returned non-zero exit status 2.


python3: can't open file '/torch_qnn/q_ops.py/torch_qnn/q_layers.pyqlstm.py': [Errno 2] No such file or directory


In [71]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import qlstm
class SensorDataset(Dataset):
    """
    Custom Dataset class for loading sensor data from CSV files.

    Args:
        data_dir (str): Path to the directory containing the CSV files.
        transform (callable, optional): Optional transform to apply to each sample.
    """
    def __init__(self, data_dir, transform=None):
        """
        Initialize the SensorDataset.
    
        Args:
            data_dir (str): Path to the directory containing the CSV files.
            transform (callable, optional): Optional transform to be applied to each sample.
        """
        self.data_dir = data_dir
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
        self.transform = transform

    def __len__(self):
        """Returns the number of files in the dataset."""
        return len(self.files)

    def __getitem__(self, idx):
        """
        Loads a single sample from the dataset.

        Args:
            idx (int): Index of the file to load.

        Returns:
            dict: A dictionary with the features tensor and the corresponding label tensor.
                  The keys are:
                  - 'features' (torch.Tensor): The feature data from the CSV file.
                  - 'label' (torch.Tensor): The label, where [1, 0] represents 'ADL' and [0, 1] represents 'Fall'.
        """
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = pd.read_csv(file_path, delimiter=',')
        data.drop(data.columns[1], axis=1, inplace=True)

        X = data[['q0', 'q1', 'q2', 'q3']].values
        sensor_id = data['sensor_id'].values.reshape(-1, 1)  # Reshape to match X's dimensions
        sample_id = data['sample'].values.reshape(-1, 1)
    
        quaternions = data[['q0', 'q1', 'q2', 'q3']].values

        #DA SISTEMARE
        # Estrai sensor_id e sample_id come colonne separate
        sensor_id = data['sensor_id'].values
        sample = data['sample'].values

        # Label extraction (e.g., based on filename)
        y = torch.Tensor([0, 1]) if 'Fall_' in file_path else torch.Tensor([1, 0])

        # Costruisci il dizionario con i dati
        sample = {
            'features': torch.tensor(quaternions, dtype=torch.float32).clone().detach(),
            'sensor_id': torch.tensor(sensor_id, dtype=torch.int32).clone().detach(),
            'sample': torch.tensor(sample, dtype=torch.int32).clone().detach(),
            'label': y.clone().detach().int()
        }

    
        if self.transform:
            sample = self.transform(sample)

        return sample


In [72]:
train = SensorDataset(data_dir="../../data/quaternions_dataset/train", transform=None)
valid = SensorDataset(data_dir="../../data/quaternions_dataset/valid", transform=None)

In [73]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train, batch_size=128, shuffle=True)
valid_loader =  DataLoader(valid, batch_size=128, shuffle=True)

In [74]:
train.__getitem__(1)

{'features': tensor([[ 0.8234, -0.0868, -0.5587, -0.0491],
         [ 0.8323, -0.0828, -0.5470, -0.0345],
         [ 0.8450, -0.0633, -0.5303, -0.0279],
         ...,
         [-0.5333,  0.8300,  0.0186,  0.1626],
         [-0.5355,  0.8283,  0.0182,  0.1637],
         [-0.5355,  0.8283,  0.0182,  0.1637]]),
 'sensor_id': tensor([1, 1, 1,  ..., 4, 4, 4], dtype=torch.int32),
 'sample': tensor([  1,   2,   3,  ..., 296, 297, 298], dtype=torch.int32),
 'label': tensor([0, 1], dtype=torch.int32)}

In [80]:
import qlstm as ciccio
model = ciccio.QLSTM(1, 10, False, num_classes=2)

In [81]:
from torch import nn
lr = 0.0005
n_epochs = 1000
best_acc = 0
patience, trials = 100, 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
opt = torch.optim.RMSprop(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    model.train()
    for i, batch in enumerate(train_loader):
        x_batch = batch["features"].to(device)
        y_batch = batch["label"].to(device)

        opt.zero_grad()
        out = model(x_batch)
        y_batch = y_batch.type(torch.FloatTensor).to(device)
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in valid_loader:
            x_val = batch["features"].to(device)
            y_val = batch["label"].to(device)
            out = model(x_val)
            preds = torch.argmax(out, dim=1)
            y_val_indices = torch.argmax(y_val, dim=1)  
            total += y_val_indices.size(0)
            correct += (preds == y_val_indices).sum().item()
    
    acc = correct / total

    if epoch % 5 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training


RuntimeError: stack expects each tensor to be equal size, but got [1196, 4] at entry 0 and [1192, 4] at entry 1